### Positive sentences from gold standard

In [1]:
expression_gold_sent_data = [line.strip().split("\t") for line in open("../../gold_standard_sentences/GROBID_clean_without_negative_otherexpr.csv")]
expression_gold_sent_data_cleaned = [[row[0], int(row[1]), row[2]] for row in expression_gold_sent_data]
expression_gold_sent_data_3_classes = [(row[0], 1 if row[1] in [0, 3] else 0, 1 if row[1] in [0, 1, 3, 4] else 0, 1 if row[1] < 9 else 0, "GOLD") for row in expression_gold_sent_data_cleaned]

kinaseact_gold_sent_data = [line.strip().split("\t") for line in open("../../gold_standard_sentences/GROBID_clean_without_negative_kinaseact.csv")]
kinaseact_gold_sent_data_cleaned = [[row[0], int(row[1]), row[2]] for row in kinaseact_gold_sent_data]
kinaseact_gold_sent_data_3_classes = [(row[0], 1 if row[1] in [0, 3] else 0, 1 if row[1] in [0, 1, 3, 4] else 0, 1 if row[1] < 9 else 0, "GOLD") for row in kinaseact_gold_sent_data_cleaned]

### Positive and negative from 1000 random sentences

In [2]:
expression_validation_data = [line.strip().split("\t") for line in open("../../validation sentences/expression_1000.tsv")]
expression_validation_data_3_classes = [(row[1], int(row[7] == "TRUE"), int(row[8] == "TRUE"), int(row[9] == "TRUE"), "1000") for row in expression_validation_data[1:] if len(row) > 10 and row[10].lower() not in ["y", "yes", "true", "t"]][:1000]

kinaseact_validation_data = [line.strip().split("\t") for line in open("../../validation sentences/kinaseact_1000.tsv")]
kinaseact_validation_data_3_classes = [(row[1], int(row[5] == "TRUE"), int(row[6] == "TRUE"), int(row[7] == "TRUE"), "1000") for row in kinaseact_validation_data[1:] if len(row) > 5 and (len(row) < 9 or row[8] != "YES")][:1000]

### Negative sentences

In [3]:
expression_negative_data = [line.strip().split("\t") for line in open("../../negative_sentences_validated/GROBID_neg_sentences_expr.tsv")]
expression_negative_data_3_classes = [(row[1], 0, 0, 0, "NEGATIVE") for row in expression_negative_data[1:] if row[2] == "Y" and (len(row) < 5 or row[4] != "Y")][:500]

kinase_negative_data = [line.strip().split("\t") for line in open("../../negative_sentences_validated/GROBID_neg_sentences_kinase.tsv")]
kinase_negative_data_3_classes = [(row[1], 0, 0, 0, "NEGATIVE") for row in kinase_negative_data[1:] if row[2] == "YES" and (len(row) < 5 or row[4] != "YES")][:500]

### Complete datasets

In [4]:
expression_unified_ds = [*expression_gold_sent_data_3_classes, *expression_validation_data_3_classes, *expression_negative_data_3_classes]


In [5]:
kinaseact_unified_ds = [*kinaseact_gold_sent_data_3_classes, *kinaseact_validation_data_3_classes, *kinase_negative_data_3_classes]

### Mark training and testing sets

In [6]:
from random import shuffle
import math

indices_expr = [i for i in range(len(expression_unified_ds))]
shuffle(indices_expr)
training_size_expr = math.ceil(len(expression_unified_ds) * 0.6)
training_indices_expr = set(indices_expr[0:training_size_expr])
test_indices_expr = set(indices_expr[training_size_expr:])
expression_unified_ds = [(*row, "TRAINING" if idx in training_indices_expr else "TESTING") for idx, row in enumerate(expression_unified_ds)]

indices_kinase = [i for i in range(len(kinaseact_unified_ds))]
shuffle(indices_kinase)
training_size_kinase = math.ceil(len(kinaseact_unified_ds) * 0.8)
training_indices_kinase = set(indices_kinase[0:training_size_kinase])
test_indices_kinase = set(indices_kinase[training_size_kinase:])
kinaseact_unified_ds = [(*row, "TRAINING" if idx in training_indices_kinase else "TESTING") for idx, row in enumerate(kinaseact_unified_ds)]

In [7]:
with open("../../unified_dataset/expression.tsv", "w") as f:
    f.write("SENTENCE\tFULLY_CURATABLE\tPARTIALLY_CURATABLE\tRELATED_LANGUAGE\tSOURCE\tTRAINING_OR_TESTING\n")
    for line in ["\t".join(map(str, line_arr)) for line_arr in expression_unified_ds]:
        f.write(line + "\n")

with open("../../unified_dataset/kinaseact.tsv", "w") as f:
    f.write("SENTENCE\tFULLY_CURATABLE\tPARTIALLY_CURATABLE\tRELATED_LANGUAGE\tSOURCE\tTRAINING_OR_TESTING\n")
    for line in ["\t".join(map(str, line_arr)) for line_arr in kinaseact_unified_ds]:
        f.write(line + "\n")